### Mapping

In [3]:
import cv2
import cv2.aruco as aruco
import numpy as np
import math
import time
import socket
import numpy as np
from cv2 import aruco
from ikpy.chain import Chain
from ikpy.urdf import URDF
from ikpy.utils import geometry
import numpy as np

camera_index = 1
ver_angle_impact = 0.1
contraction = 1

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="ikpy.chain")
# Step 1: Function to calculate pixel distance along the y-axis between Marker 1 at origin and Marker 2
def calculate_pixel_distance(marker_y_axis_y):
    # Since Marker 1 is at (0,0), the pixel distance is simply the y-coordinate of Marker 2
    pixel_distance = abs(marker_y_axis_y)
    return pixel_distance

# Step 2: Function to adjust pixel distance with cos(phi)
def adjust_distance_with_angle(pixel_distance, angle):
    # Adjust pixel distance using cos(phi) to account for angle distortion
    adjusted_distance = pixel_distance * np.cos(angle)
    return adjusted_distance

# Step 3: Get robot's coordinate system for origin (O) and reference point (R)
def get_robot_coordinates():
    print("Enter the coordinates of the camera origin in the robot's coordinate system (Ox, Oy):")
    Ox, Oy = map(float, input("Ox Oy: ").split())
    print("Enter the coordinates of the reference point in the robot's coordinate system (Rx, Ry):")
    Rx, Ry = map(float, input("Rx Ry: ").split())
    return (Ox, Oy), (Rx, Ry)

# Step 4: Calculate the proportionality constant c
def calculate_proportionality_constant(robot_distance, pixel_distance):
    # c = d / p
    c = robot_distance / pixel_distance
    return c

# Step 5: Calculate theta, the angle between robot's y-axis and camera's y-axis
def calculate_theta(Ox, Oy, Rx, Ry):
    # Calculate the angle using atan2 for correct quadrant handling
    delta_x = Rx - Ox
    delta_y = Ry - Oy
    theta = np.arctan2(delta_y, delta_x)
    return theta-np.pi/2

# Step 6: Transformation function to convert any point (x, y) in camera coordinates to robot coordinates
def camera_to_robot_coordinates(O, c, theta, point_in_camera):
    # Ensure input types are correct
    O = np.array(O, dtype=np.float64)  # Origin of robot coordinates
    point_in_camera = np.array(tuple(contraction * x for x in point_in_camera), dtype=np.float64)  # Camera coordinates

    # Rotation matrix
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ], dtype=np.float64)

    # Scale and rotate the point
    adjusted_point = c * np.dot(rotation_matrix, point_in_camera)

    # Translate to the robot's coordinate system by adding origin coordinates (Ox, Oy)
    point_in_robot = O + adjusted_point

    return point_in_robot



In [4]:

# Initialize the camera
cap = cv2.VideoCapture(camera_index)  # Change to 1 or 2 if multiple cameras are connected

if not cap.isOpened():
    print("Error: Could not open camera. Check permissions or try a different index.")
    exit()

# Define the dictionary and parameters for ArUco detection
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)
parameters = aruco.DetectorParameters()
parameters.minMarkerPerimeterRate = 0.01  # Increase sensitivity for smaller markers
parameters.adaptiveThreshWinSizeMin = 3   # Min window size for adaptive thresholding
parameters.adaptiveThreshWinSizeMax = 25  # Max window size for adaptive thresholding
parameters.adaptiveThreshWinSizeStep = 5  # Step size for adaptive thresholding
parameters.polygonalApproxAccuracyRate = 0.05  # Improves marker detection near edges
 
# Get the frame resolution to determine camera center
ret, frame = cap.read()
if not ret:
    print("Failed to capture initial image for resolution check.")
    cap.release()
    exit()

height, width = frame.shape[:2]
camera_center = (width // 2, int(height * 4.5 / 5))


# Dummy camera parameters (required for pose estimation)
camera_matrix = np.array([[800, 0, width // 2], [0, 800, height // 2], [0, 0, 1]], dtype=float)
dist_coeffs = np.zeros((4, 1))

# Dictionary to store marker data dynamically
marker_data = {}

# Start the video capture loop
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    # Create a separate grayscale frame for clean marker detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect markers on the clean grayscale frame
    corners, ids, rejected = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # Overlay for drawing axes and detected marker information
    overlay = frame.copy()
    alpha = 0.5  # Transparency factor for overlay

    # Draw semi-transparent axes on the overlay
    cv2.line(overlay, (0, camera_center[1]), (width, camera_center[1]), (255, 0, 0), 3)  # x-axis
    cv2.line(overlay, (camera_center[0], 0), (camera_center[0], height), (0, 255, 0), 3)  # y-axis

    # If markers are detected
    if ids is not None:
        for i, corner in enumerate(corners):
            # Estimate pose of the marker to get its rotation and translation vectors
            rvec, tvec, _ = aruco.estimatePoseSingleMarkers(corner, 0.05, camera_matrix, dist_coeffs)

            # Convert rotation vector to rotation matrix
            rotation_matrix, _ = cv2.Rodrigues(rvec[0])

            # Extract pitch and roll, adding a small epsilon to avoid division by zero
            pitch = math.degrees(math.atan2(-rotation_matrix[2, 0], np.sqrt(rotation_matrix[2, 1]**2 + rotation_matrix[2, 2]**2 + 1e-6)))
            roll = math.degrees(math.atan2(rotation_matrix[2, 1], rotation_matrix[2, 2]))

            # Calculate the vertical angle (angle between marker surface normal and camera viewing vector)
            marker_normal = rotation_matrix[:, 2]  # Z-axis of the marker
            camera_viewing_vector = np.array([0, 0, -1])
            cos_theta = np.dot(marker_normal, camera_viewing_vector) / (np.linalg.norm(marker_normal) * np.linalg.norm(camera_viewing_vector))
            vertical_angle = np.degrees(np.arccos(cos_theta))

            # Calculate the center of the marker relative to the camera center
            center_x = int(np.mean(corner[0][:, 0]))
            center_y = int(np.mean(corner[0][:, 1]))
            
            relative_x = (center_x - camera_center[0])
            relative_y = (camera_center[1] - center_y)  # Inverting y-axis for upward positive direction

            # Draw detected markers on the overlay
            aruco.drawDetectedMarkers(overlay, [corner], ids[i])

            # Draw the coordinate axes on the marker
            cv2.drawFrameAxes(overlay, camera_matrix, dist_coeffs, rvec[0], tvec[0], 0.1)  # Axis length can be adjusted

            # Text information with different colors and larger font size
            info_texts = [
                ("Pitch: {:.2f}°".format(pitch), (255, 255, 0)),   # Yellow for Pitch
                ("Roll: {:.2f}°".format(roll), (0, 0, 255)),       # Red for Roll
                ("Vertical Angle: {:.2f}°".format(vertical_angle), (0, 255, 255)),  # Cyan for Vertical Angle
                ("Pos: ({}, {})".format(relative_x, relative_y), (255, 255, 255))  # White for Position
            ]

            # Determine the position of the box to avoid covering the marker
            box_width, box_height = 250, 150
            box_x = center_x + 20 if center_x < width - box_width - 20 else center_x - box_width - 20
            box_y = center_y + 20 if center_y < height - box_height - 20 else center_y - box_height - 20

            # Draw the opaque black background box for information
            cv2.rectangle(overlay, (box_x, box_y), (box_x + box_width, box_y + box_height), (0, 0, 0), -1)

            # Display all information in the box with specified colors and larger font size
            for j, (text, color) in enumerate(info_texts):
                cv2.putText(overlay, text, (box_x + 10, box_y + 35 + j * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            # Store the marker data dynamically in the dictionary
            marker_id = ids[i][0]
            marker_data[marker_id] = {
                "pitch": pitch,
                "roll": roll,
                "vertical_angle": vertical_angle * ver_angle_impact,
                "position": (relative_x, relative_y)
            }

    # Blend the overlay with the original frame to apply semi-transparent axes and marker info
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    # Display the frame with overlay
    cv2.imshow('ArUco Marker Detection with Orientation', frame)

    # Capture marker data on pressing 'c'
    if cv2.waitKey(1) & 0xFF == ord('c'):
        # Print captured marker data and find the maximum y-coordinate and vertical angle
        print("\n\nCaptured marker data:")

        RCy = -float('inf')
        max_marker_id = None
        max_position = None
        vertical_angle_for_max_y = None  # To store vertical angle for the max y marker

        for marker_id, data in marker_data.items():
            print(f"Marker ID: {marker_id}, Data: {data}")

            # Extract the y-coordinate from the position
            relative_y = data["position"][1]
            if relative_y > RCy:
                RCy = relative_y
                max_marker_id = marker_id
                max_position = data["position"]
                vertical_angle_for_max_y = data["vertical_angle"]  # Store vertical angle

        # Print the maximum y-coordinate and corresponding marker information
        print(f"Maximum y-coordinate (RCy): {RCy} for Marker ID: {max_marker_id} at Position: {max_position}")
        print(f"Vertical Angle for Marker with Max Y: {vertical_angle_for_max_y}")
        marker_y_axis_y = RCy

        user_input = input("\nDo you want to go ahead? (y/n): ").strip().lower()

        if user_input == "y":
                print("\nProceeding to send_angles to the ROBOT!\n\n")
                break
                # Add code to go ahead with the action here
        elif user_input == "n":
                print("Stopping.\n")
                continue  # Exit the loop if they choose not to go ahead
        else:
                print("Invalid input, please answer with 'yes' or 'no'.")
                continue
        
    # Exit on pressing 'q'
    elif cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


# 1. Calculate pixel distance between the origin and the y-axis marker
pixel_distance = calculate_pixel_distance(marker_y_axis_y)
print("Pixel distance between markers:", pixel_distance)

# 2. Adjust for angle with cos(phi)
phi = np.radians(vertical_angle_for_max_y)  # Example angle (in radians) between marker's axis and camera line of sight
adjusted_pixel_distance = adjust_distance_with_angle(pixel_distance, phi) #adjust_distance_with_angle(pixel_distance, phi)
print("Adjusted pixel distance:", adjusted_pixel_distance,"\n")

# 3. Get robot's coordinates for origin (O) and reference (R)
while True:
    try:
        O, R = get_robot_coordinates()
        break
    except:
        continue

print(f"Camera Origin in robot's coordinate system: O(0,0)== {O}")
print(f"Camera Reference in robot's coordinate system: R(0,{marker_y_axis_y})== {R}\n")

# 4. Calculate the robot distance between O and R
robot_distance = np.linalg.norm(np.array(R) - np.array(O))
print(f"Distance between points (robot_distance): {robot_distance}")

# 5. Calculate proportionality constant c
c = calculate_proportionality_constant(robot_distance, adjusted_pixel_distance)
print("Proportionality constant (c):", c)

# 6. Calculate theta, the angle between robot's y-axis and camera's y-axis
theta = calculate_theta(*O, *R)
print("Angle theta (radians) between robot's y-axis and camera's y-axis:", theta*(180/np.pi))

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

### Maze

In [ ]:
import cv2
import numpy as np
import time


# Function to handle rectangle selection and image processing
def process_image(image):
    # Let the user select a rectangular area
    r = cv2.selectROI("Select the area to keep", image)
    cv2.destroyWindow("Select the area to keep")
    
    # Create a white background image
    output_image = np.ones_like(image) * 255
    
    # Copy the selected rectangle region from the original image to the white background
    x, y, w, h = r
    output_image[y:y+h, x:x+w] = image[y:y+h, x:x+w]
    
    # Save the processed image
    cv2.imwrite("captured_img.png", output_image)
    
    # Show the processed image
    cv2.imshow("Processed Image", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Solves the maze
def solve_image(img):
    # Convert the image to grayscale (black and white)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to get a pure black and white image (invert the colors)
    ret, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV) 
    # Increase the kernel size for stronger blurring
    blurred = cv2.GaussianBlur(gray, (15, 15), 0)
    cv2.imshow('Threshold 1', thresh)  # Show the first thresholded image

    # Find contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    dc = cv2.drawContours(thresh.copy(), contours, 0, (0, 0, 255), 10)  # Red color with thicker lines
    cv2.imshow('Contours 1', dc)  # Show the first contours
    
    # Apply second thresholding to clean up the contours
    ret, thresh = cv2.threshold(dc, 240, 255, cv2.THRESH_BINARY)
    #cv2.imshow('Threshold 2', thresh)  # Show the second thresholded image

    # Dilation
    # Kernel size for dilation and erosion
    ke = kernel_defined
    kernel = np.ones((ke, ke), np.uint8)
    dilation = cv2.dilate(thresh, kernel, iterations=1)
    #cv2.imshow('Dilation', dilation)  # Show dilation result

    # Erosion
    erosion = cv2.erode(dilation, kernel, iterations=1)
    #cv2.imshow('Erosion', erosion)  # Show erosion result

    # Find differences between dilation and erosion
    diff = cv2.absdiff(dilation, erosion)
    #cv2.imshow('Difference', diff)  # Show the difference between dilation and erosion

    # Erosion the diff image again
    ke1 = int(round(ke / 2.5, 0))  # Convert to integer explicitly
    kernel = np.ones((ke1, ke1), np.uint8)
    erosion_diff = cv2.erode(diff, kernel, iterations=1)
    #cv2.imshow('Erosion Diff', erosion_diff)  # Show the eroded difference image

    # Invert the mask
    mask_inv = cv2.bitwise_not(erosion_diff)
    cv2.imshow('Mask', mask_inv)  # Show the inverted mask

    # Split the channels of the maze image
    b, g, r = cv2.split(img)

    # Mask out the green and red color from the solved path
    r = cv2.bitwise_and(r, r, mask=mask_inv)

    b = cv2.bitwise_and(b, b, mask=mask_inv)

    # Merge the channels to display the solved maze
    res = cv2.merge((b, g, r))
    cv2.imshow('Solved Maze', res)  # Show the solved maze

    cv2.waitKey(6000)  # Wait for a key press
    # Close all windows after any key is pressed
    cv2.destroyAllWindows()
    return mask_inv

# Opens the camera to capture the maze
def open_camera():
    # Capture the live video feed
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Show the live video feed
        cv2.imshow('Live Feed', frame)
        
        # Wait for keypress: 'c' to capture image, 'q' to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            cv2.destroyWindow("Live Feed")
            captured_image = frame.copy()
            #cv2.imshow("Captured Image", captured_image)
            cv2.waitKey(1)  # Wait a moment for the user to see the captured image
            return captured_image
        elif key == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Marks coordinates and orders them to cross the maze properly
def process_solution(filename, direction=-1, start_from_point=1):
    # Load the image
    img = cv2.imread(filename + '.png')

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur for noise reduction
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply binary threshold to isolate the black line/loop
    _, binary = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV)

    # Skeletonize using OpenCV's morphological operations
    skeleton = np.zeros_like(binary)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    temp_binary = binary.copy()

    while cv2.countNonZero(temp_binary) > 0:
        eroded = cv2.erode(temp_binary, element)
        temp = cv2.dilate(eroded, element)
        temp = cv2.subtract(temp_binary, temp)
        skeleton = cv2.bitwise_or(skeleton, temp)
        temp_binary = eroded

    # Convert skeleton to contours for further processing
    contours, _ = cv2.findContours(skeleton, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Focus on the largest contour (assuming it's the main loop)
    contour = max(contours, key=cv2.contourArea)
    
    # Approximate the contour to reduce noise and focus on corners
    epsilon = 0.01 * cv2.arcLength(contour, True)  # Adjust epsilon for more/less simplification
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Convert to a more uniform format (list of (x, y) tuples)
    approx = [tuple(point[0]) for point in approx]

    # Find the top-left corner to start numbering
    top_left_corner = min(approx, key=lambda p: (p[0] + p[1]))  # Smallest (x + y) is the top-left corner
    start_index = approx.index(top_left_corner)

    # Reorder corners to start from the top-left corner
    ordered_corners = approx[start_index:] + approx[:start_index]

    # Adjust the start point (convert from 1-based index to 0-based index)
    start_index = start_from_point - 1

    # Reorder corners to start from the specified point
    ordered_corners = ordered_corners[start_index:] + ordered_corners[:start_index]

    # Reverse the order of points based on direction
    if direction == -1:
        ordered_corners = ordered_corners[::-1]

    # Prepare the corner dictionary
    corner_dict = {}

    # Image size for coordinate system
    height, width = skeleton.shape
    center_x, center_y = width // 2, int(height * 4.5 / 5)

    img_with_corners = cv2.cvtColor(skeleton, cv2.COLOR_GRAY2BGR)

    for i, (x, y) in enumerate(ordered_corners, 1):
        # Adjust the coordinates to have the origin at the center
        adjusted_x = x - center_x
        adjusted_y = -(y - center_y)  # Flip y to make it positive upwards

        # Draw the corner on the image
        cv2.circle(img_with_corners, (x, y), 5, (0, 0, 255), -1)  # Red dot
        # Annotate the corner with a serial number
        cv2.putText(img_with_corners, f'{i}', (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
        # Annotate the adjusted coordinates (in the center-based coordinate system)
        cv2.putText(img_with_corners, f'({adjusted_x}, {adjusted_y})', (x + 5, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)
        
        # Add the point with adjusted coordinates to the dictionary as 'coords'
        corner_dict[f"Serial_{i}"] = [adjusted_x, adjusted_y]

    # Overlay the coordinate axes (x-axis and y-axis)
    axis_color = (255, 0, 0)  # Blue for the axes
    thickness = 2

    # Draw x-axis
    cv2.line(img_with_corners, (0, center_y), (width, center_y), axis_color, thickness)

    # Draw y-axis
    cv2.line(img_with_corners, (center_x, 0), (center_x, height), axis_color, thickness)

    # Display the image with the points, their serial numbers, and the axes
    cv2.imshow('Simplified Curve with Corners and Axes', img_with_corners)
    cv2.imwrite("final_Corners_With_Axes.png", img_with_corners)
    print("Saved the solution image into \"final_Corners_With_Axes.png\"")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return corner_dict



In [ ]:
# This captures the image
cap = cv2.VideoCapture(camera_index)
captured_image = open_camera()
if captured_image is not None:
    # Process the captured image
    process_image(captured_image)
time.sleep(1)
cap.release() 
cv2.destroyAllWindows()

In [ ]:
# Maze solution
kernel_defined =70 # Change this for better maze solution detection

# This finds solution
img = cv2.imread('captured_img.png')
solution = solve_image(img)
cv2.imshow('Final', solution)
cv2.waitKey(2000000)
cv2.imwrite("solution.png", solution)
print("Saved the solution image into \"solution.png\"")

 # Releases the video capture object
cv2.destroyAllWindows()  # Closes any open OpenCV windows

In [ ]:
O = (-384.83, -462.958)
c= 0.44737345224705083
theta = 2.642136069702122 * np.pi/180

In [ ]:
# This marks coordinates and orders them
filename = 'solution'  # Replace with your image fqilename without extension
direction = -1  # Set to 1 for anticlockwise, -1 for clockwise
start_from_point = 1  # Set the starting point index (1-based index)

corner_dict = process_solution(filename, direction, start_from_point)

print("Corner Dictionary:")
for serial, coords in corner_dict.items():
    print(f"{serial}: {coords}")



print("\nActual Points:")
actual_points = {}

# Iterate through each serial and its corresponding coordinates
for serial, coords in corner_dict.items():
    # Extract the coordinates from the dictionary
    point_in_camera = coords
    # Transform the coordinates from camera to robot frame
    point_in_robot = camera_to_robot_coordinates(O, c, theta, point_in_camera)
    # Transformation matrix

    # transformation_matrix = np.array([[-1, 0],
    #                                 [0, -1]])
    # # Perform matrix multiplication
    # point_in_robot = transformation_matrix @ point_in_robot
    
    # Convert the np.float64 values to native Python float and store them in the new dictionary
    actual_points[serial] =  [round(float(point_in_robot[0]),2), round(float(point_in_robot[1]),2)]
    # Prepare the string output in MATLAB containers.Map format

for serial, coords in actual_points.items():
    print(f"{serial}= {coords}")


### Inverse

In [ ]:
import numpy as np
from ikpy.chain import Chain
from scipy.optimize import fsolve, root
import sounddevice as sd
import sympy as sp
import cv2
import cv2.aruco as aruco
import math
import time
import socket
from ikpy.urdf import URDF
from ikpy.utils import geometry


# Mathemaatically these calcualtions can never give any error. It's always 100000000% correct.
error_tolerance = 0.001      # Tolerance for differences in inverse and forward. (in mm)
z= 60                      # Height of the end effector form the table top. (in mm)

# Alert sound
def urgent_buzzer(duration=0.4, freq=400, fs=44100):
    t = np.linspace(0, duration, int(fs * duration), endpoint=False)
    buzzer = 0.5 * np.sin(6 * np.pi * freq * t) * (np.sin(12 * np.pi * 5 * t) > 0)  # Modulated "on-off"
    sd.play(buzzer, fs)
    sd.wait()

# Define the symbolic variables
Px, Py, Pz, t4, t5, t6 = sp.symbols('Px Py Pz t4 t5 t6')

# H14 Matrix
H14 = sp.Matrix([[Px / sp.sqrt(Px**2 + Py**2),  0, -Py / sp.sqrt(Px**2 + Py**2), Px],
                    [Py / sp.sqrt(Px**2 + Py**2),  0,  Px / sp.sqrt(Px**2 + Py**2), Py],
                    [0, -1, 0, Pz],
                    [0, 0, 0, 1]])

# H45 Matrix
H45 = sp.Matrix([[sp.cos(t4),  0, -sp.sin(t4), 0],
                    [sp.sin(t4),  0,  sp.cos(t4), 0],
                    [0, -1, 0, 219 / 2],
                    [0, 0, 0, 1]])

# H56 Matrix
H56 = sp.Matrix([[sp.cos(t5), 0, sp.sin(t5), 0],
                    [sp.sin(t5), 0, -sp.cos(t5), 0],
                    [0, 1, 0, 107],
                    [0, 0, 0, 1]])

# H6e Matrix
H6e = sp.Matrix([[sp.cos(t6), -sp.sin(t6), 0, 0],
                    [sp.sin(t6), sp.cos(t6), 0, 0],
                    [0, 0, 1, 381 / 5],
                    [0, 0, 0, 1]])

# Compute the total transformation matrix H_total
H_total = H14 * H45 * H56 * H6e

translation_vector = H_total[0:3, 3]

Sx = translation_vector[0]
Sy = translation_vector[1]
Sz = translation_vector[2]
        
# Extract the translational vector (last column of H_total)
translation_vector = H_total[0:3, 3]


def get_link3endsimple(Kx_val, Ky_val, Kz_val, t4_val=-90, t5_val=-90):  
    # t4_val this is fake angle when we consider the robot to be link1 connected to link 4 directly.
    
    # Define the symbolic variables
    Px, Py, Pz, t4, t5, t6 = sp.symbols('Px Py Pz t4 t5 t6')

    # Convert angles t4, t5 to radians
    t4_val = t4_val * sp.pi / 180  # Convert to radians
    t5_val = t5_val * sp.pi / 180  # Convert to radians

    # Set up the system of equations
    eq1 = sp.Eq(Sx, Kx_val)
    eq2 = sp.Eq(Sy, Ky_val)
    eq3 = sp.Eq(Sz, Kz_val)

    # Substitute the t4, t5 values into the equations
    eq1_subs = eq1.subs({t4: t4_val, t5: t5_val})
    eq2_subs = eq2.subs({t4: t4_val, t5: t5_val})
    eq3_subs = eq3.subs({t4: t4_val, t5: t5_val})

    # Set initial guesses for Px, Py, Pz
    initial_guess = (100, 100, 100)  # Initial guess for (Px, Py, Pz)

    # Use numerical solving (nsolve) for all three equations with initial guesses
    solution = sp.nsolve([eq1_subs, eq2_subs, eq3_subs], (Px, Py, Pz), initial_guess)

    # Convert the solution to numerical values using evalf
    Px = float(solution[0])  # Convert to numeric (float)
    Py = float(solution[1])
    Pz = float(solution[2])

    # Return the solved values as numeric
    return Px, Py, Pz



# This then calcualtes all teh joint angles.
def get_inverse(x, y, z=133.8, t4_val=-90, t5_val=-90):
    
    Px, Py, Pz = get_link3endsimple(x, y, z, t4_val, t5_val)
    #print(Px,Py,Pz)

    t1 = np.arctan2(Py, Px) * 180 / np.pi
    dz = Pz-210 # 210 is link1 length, we need to be on the link1 heihgt plane to solve triangle base and extra_t2
    # The base for triangle formed by links 2 and 3
    base = np.sqrt(Px**2 + Py**2 + (dz)**2)

    base2 = np.sqrt(Px**2 + Py**2) # Base with no z componenet, it helps to know the extraa_t2 which is made by sides base2 and Pz.
    #print(base2)
    #print(Pz-210)
    extra_t2 = np.degrees(np.arctan2(dz, base2))
    #print(extra_t2)

    # Step 2: Use the Law of Cosines to find the angle at the vertex (theta)
    side = 250  # Length of the equal sides (250)
    theta = np.arccos((2 * side**2 - base**2) / (2 * side**2))  # Angle in radians

    alpha_actual = np.degrees((np.pi - theta)/2)
    #print("alpha actual: ", alpha_actual)
    # Base angles (alpha) for link 2 and 3

    # Convert to degrees
    theta_deg = np.degrees(theta)

    angles = [
        t1,
        -1 * (alpha_actual+extra_t2),
        180 - theta_deg,
        -1 * (alpha_actual-extra_t2)+t4_val,
        t5_val,
        0
    ]
    if any(np.isnan(angle) for angle in angles): print("NAAAAAAAAAAAAAAAAAANNNNNNNNN"); urgent_buzzer()

    if (180 - theta_deg) > 149:
        urgent_buzzer()

    robot_chain = Chain.from_urdf_file("my_pro600.urdf")
    new_angles = [0.0] + angles
    angles_rad = np.radians(new_angles)
    fk_result_1 = robot_chain.forward_kinematics(angles_rad)
    fk_result_1_rounded = np.round(fk_result_1, 8)
    print(fk_result_1_rounded)

    target_position = np.array([x, y , z])
    diff_x = round((fk_result_1_rounded[0, 3]*1000 - target_position[0]), 3)
    diff_y = round((fk_result_1_rounded[1, 3]*1000 - target_position[1]), 3)
    diff_z = round((fk_result_1_rounded[2, 3]*1000 - target_position[2]), 3)
    if abs(diff_x) > error_tolerance or abs(diff_y) > error_tolerance or abs(diff_z) > error_tolerance:
        print("\n\033[1;34m********************BADDDDD********************\033[0m\n")
        urgent_buzzer()
    print(f"Differences for solution 1: \033[1;31mDx = {diff_x}, Dy = {diff_y}, Dz = {diff_z}\033[0m")

    return angles



joint_angles_dict = {}
 
# Convert dictionary to a list of items for iteration with index
actual_points_items = list(actual_points.items())

# Iterate over the dictionary, skipping the first and last items
for idx, (serial, coords) in enumerate(actual_points_items):
    if idx == 0 :
        continue  # Skip the first and last items

    point_in_robot = coords
    x, y = point_in_robot

    print(f"\n{serial} Coordinates: \033[1;93m({round(x,3)}, {round(y,3)}, {round(z,3)})\033[0m")
    #print(f"Distance: {(Px**2 + Py**2)**0.5}")

    # Call your inverse kinematics function
    joint_angles = get_inverse(x, y, z)
    joint_angles_dict[serial] = joint_angles  # Store the joint angles under the serial number


print("\n\n")
for serial, joint_angles in joint_angles_dict.items():
    print(f"{serial}: {joint_angles}")
    i=1
    for _ in joint_angles:
        print(f"t{i}v= {_};")
        i+=1
    print("/n")


In [ ]:
result = get_inverse(x=0, y=350, z=80, t4_val=45, t5_val=-90)
print(result)

### Send Angles

In [ ]:
import socket
import sounddevice as sd
import numpy as np

# Define your server IP and port
SERVER_IP = "192.168.1.159"
SERVER_PORT = 5001

# Sound alert
def urgent_buzzer(duration=0.64, freq=600, fs=44100):
    t = np.linspace(0, duration, int(fs * duration), endpoint=False)
    buzzer = 0.5 * np.sin(2 * np.pi * freq * t) * (np.sin(4 * np.pi * 4 * t) > 0)  # Modulated "on-off"
    sd.play(buzzer, fs)
    sd.wait()

# Function to send a TCP packet to the robot
def send_tcp_packet(server_ip, server_port, message):
    try:
        # Create a TCP socket
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        # Connect to the server
        client_socket.connect((server_ip, server_port))
        print(f"Connected to {server_ip}:{server_port}")

        # Send the message
        client_socket.sendall(message.encode('utf-8'))
        print(f"Sent: {message}")

        # Optionally receive a response (if server sends one)
        response = client_socket.recv(1024).decode('utf-8')
        print(f"Received: {response}")

    except socket.error as e:
        print(f"Error: {e}")

    finally:
        # Close the connection
        client_socket.close()
        #print("Connection closed.")

# Function to send the joint angles
def send_point_angles(solutions):

    i=1
    for angles in solutions:
        # Format angles as a string with 2 decimal places for TCP message
        rounded_angles = [round(angle, 2) for angle in angles]
        print(rounded_angles)
        angle_message = f"set_angles({', '.join(map(str, rounded_angles))}, 1000)"

        # Send the message to the robot
        send_tcp_packet(SERVER_IP, SERVER_PORT, angle_message)
        if i==1:
            print("Waiting 2s before starting the MAZE!\n")
            time.sleep(5)
        i+=1
        time.sleep(0)

# Sending angles for each coordinate:
urgent_buzzer()
time.sleep(1)
send_point_angles(solutions)
